# Clustering Crypto

In [2]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [3]:
# Load the crypto_data.csv dataset.
# YOUR CODE HERE

file_path = "crypto_data.csv"
df_crypto = pd.read_csv(file_path)
df_crypto.head(5)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [4]:
# Keep all the cryptocurrencies that are being traded.
df_crypto = df_crypto.set_index("Unnamed: 0")
df_crypto.index.name = None
df_crypto.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [5]:
# Keep all the cryptocurrencies that have a working algorithm.
# YOUR CODE HERE

df_crypto=df_crypto[df_crypto['IsTrading'] == True]
df_crypto.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [6]:
# Remove the "IsTrading" column. 

df_crypto.drop(columns=["IsTrading"], inplace = True)
df_crypto.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [23]:
# Remove rows that have at least 1 null value.
# YOUR CODE HERE
# find null values
for column in df_crypto.columns:
    print(f"Column {column} has {df_crypto[column].isnull().sum()}")
#    print(f"Column {column} has {df_crypto[column].isnull().sum()})

Column CoinName has 0
Column Algorithm has 0
Column ProofType has 0
Column TotalCoinsMined has 459
Column TotalCoinSupply has 0


In [24]:
#drop the null rows
df_crypto=df_crypto.dropna()

In [25]:
# Keep the rows where coins are mined.
# YOUR CODE HERE 
#same as null coins mines....
df_crypto = df_crypto[df_crypto['TotalCoinsMined'] > 0]
df_crypto

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [26]:
# Create a new DataFrame that holds only the cryptocurrencies names.
# YOUR CODE HERE
df_coinname = pd.DataFrame(df_crypto['CoinName'])
df_coinname.head()


,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [27]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
# YOUR CODE HERE

df_crypto.drop(columns=["CoinName"], inplace=True)
df_crypto.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [28]:
# Use get_dummies() to create variables for text features.
# YOUR CODE HERE
X = pd.get_dummies(df_crypto, columns=["Algorithm","ProofType"])

In [29]:
X.head(5)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# Standardize the data with StandardScaler().
# YOUR CODE HERE

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled


array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [31]:
# Using PCA to reduce dimension to three principal components.
# YOUR CODE HERE
# Initialize PCA model
pca = PCA(n_components=3)

# Get three principal components for the crypto data.
crypto_pca = pca.fit_transform(X_scaled)
crypto_pca

array([[-0.35383495,  0.82914003, -0.57623705],
       [-0.33721182,  0.82899827, -0.57665604],
       [ 2.31116262,  1.59669161, -0.66610423],
       ...,
       [ 0.34880509, -2.09203238,  0.49410184],
       [-0.14088282, -2.0979725 ,  0.41318701],
       [-0.30997481,  0.67307747, -0.30057789]])

In [33]:
# Create a DataFrame with the three principal components.
# YOUR CODE HERE

pcs_df = pd.DataFrame(crypto_pca, columns=['PrincipleComponent1', 'PrincipleComponent2', 'PrincipleComponent3'], index=df_crypto.index)
pcs_df

,PrincipleComponent1,PrincipleComponent2,PrincipleComponent3
42,-0.353835,0.829140,-0.576237
404,-0.337212,0.828998,-0.576656
1337,2.311163,1.596692,-0.666104
BTC,-0.120602,-1.178107,0.256766
ETH,-0.127822,-1.812884,0.420230
...,...,...,...
ZEPH,2.505419,1.125772,-0.255151
GAP,-0.351883,0.829006,-0.576253
BDX,0.348805,-2.092032,0.494102
ZEN,-0.140883,-2.097973,0.413187


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [38]:
# Create an elbow curve to find the best value for K.
# YOUR CODE HERE
inertia = []
k = list(range(1, 11))

# Looking for the best K
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

C:\Users\Range\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  "KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [42]:
# Initialize the K-Means model.
# YOUR CODE HERE
model=KMeans(n_clusters=4, random_state=5).fit(X_scaled)


# Fit the model
# YOUR CODE HERE

#model.fit(pcs_df)

# Predict clusters
# YOUR CODE HERE

predictions=model.predict(X_scaled)
print(predictions)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [43]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
# YOUR CODE HERE
clustered_df = pd.concat([df_crypto, pcs_df], axis=1, join='inner')

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
# YOUR CODE HERE
clustered_df['CoinName'] = df_coinname['CoinName']

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# YOUR CODE HERE
clustered_df['Class'] = predictions

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PrincipleComponent1,PrincipleComponent2,PrincipleComponent3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.353835,0.829140,-0.576237,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.337212,0.828998,-0.576656,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.311163,1.596692,-0.666104,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.120602,-1.178107,0.256766,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.127822,-1.812884,0.420230,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.172362,-1.183106,0.022692,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.395736,1.202856,-0.500546,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.128521,-1.981653,0.475386,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.126266,-1.812997,0.420219,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.140882,-2.097972,0.413187,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [55]:
from matplotlib import pyplot as plt

In [73]:
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PrincipleComponent1,PrincipleComponent2,PrincipleComponent3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.353835,0.829140,-0.576237,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.337212,0.828998,-0.576656,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.311163,1.596692,-0.666104,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.120602,-1.178107,0.256766,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.127822,-1.812884,0.420230,Ethereum,0


In [75]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE
fig = px.scatter_3d(
    clustered_df,
    x="PrincipleComponent1",
    y="PrincipleComponent2",
    z="PrincipleComponent3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name="CoinName",
    hover_data=["Algorithm"]
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [76]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE
clustered_df.hvplot.table(sortable=True, selectable=True)

:Table   [Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PrincipleComponent1,PrincipleComponent2,PrincipleComponent3,CoinName,Class]

In [77]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE
print(f"There are {clustered_df['CoinName'].count()} tradable cryptocurrencies.")


There are 532 tradable cryptocurrencies.


In [78]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE
coins = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']]
coins_scaled = MinMaxScaler().fit_transform(coins)
coins_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [79]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE
plot_df = pd.DataFrame(coins_scaled, columns=['TotalCoinSupply', 'TotalCoinsMined'], index=clustered_df.index)


# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE
plot_df['CoinName'] = clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE
plot_df['Class'] = clustered_df['Class']

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0
ZEC,2.100000e-05,0.000007,ZCash,0


In [80]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
plot_df.value_counts(['Class'])

Class
0        529
1          1
2          1
3          1
dtype: int64